In [ ]:

pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 52.8 MB/s eta 0:00:00


In [ ]:
import pulp as lp
import pandas as pd

In [ ]:
# Store results for all countries
optimization_results = {}

# Load the processed data from CSV
final_df = pd.read_csv("Final_Processed_Data.csv")

# Iterate through each country
for country in final_df["Country"].unique():
    # Filter data for the specific country
    country_df = final_df[final_df["Country"] == country]

    # Extract parameters
    advertiser_data = country_df[["Month", "Advertisers", "Avg_Budget"]].reset_index(drop=True)
    initial_agents = country_df["Existing_Agent_Count"].iloc[0]
    annual_salary = country_df["Annual_Agent_Salary_USD"].iloc[0]
    monthly_salary = annual_salary / 12

    # Constants
    firing_cost_perc = 0.4
    max_adv_per_agent = 10
    expected_uplift = 0.135
    n_months = len(advertiser_data)

    # Initialize PuLP problem
    prob = lp.LpProblem(f"Dynamic_Staffing_Optimization_{country}", lp.LpMinimize)

    # Decision Variables
    months = list(range(n_months))
    A = lp.LpVariable.dicts("Available_Agents", months, lowBound=0, cat='Integer')
    N = lp.LpVariable.dicts("New_Hires", months, lowBound=0, cat='Integer')
    F = lp.LpVariable.dicts("Fired", months, lowBound=0, cat='Integer')
    S = lp.LpVariable.dicts("Assigned_Advertisers", months, lowBound=0, cat='Integer')
    W = lp.LpVariable.dicts("Waiting_Pool", months, lowBound=0, cat='Integer')
    D = lp.LpVariable.dicts("Dropped_Advertisers", months, lowBound=0, cat='Integer')
    G = lp.LpVariable.dicts("Graduated_Advertisers", months, lowBound=0, cat='Integer')

    # Initial Conditions
    prob += A[0] == initial_agents

    # Constraints
    W_prev = 0  # Initial waiting pool
    S_prev = 0  # Initial assigned advertisers

    for m in months:
        if m == 0:
            T = A[m]
        else:
            T = A[m] + N[m-1] - F[m-1]

        prob += S[m] <= T * max_adv_per_agent
        prob += W[m] == W_prev + advertiser_data["Advertisers"][m] - S[m] - D[m]
        prob += W[m] >= 0

        if m >= 2:
            prob += D[m] == W[m-2]
            prob += G[m] == S[m-2]
        else:
            prob += D[m] == 0
            prob += G[m] == 0

        prob += N[m] <= A[m]
        prob += F[m] <= A[m]
        prob += T * 10 >= S[m] + S_prev

        if m < n_months - 1:
            prob += A[m+1] == A[m] + N[m] - F[m] + G[m]

        W_prev = W[m]
        S_prev = S[m]

    # Objective Function
    total_cost = lp.lpSum([
        A[m] * monthly_salary + F[m] * firing_cost_perc * annual_salary
        - S[m] * advertiser_data["Avg_Budget"][m] * expected_uplift
        + W[m] * (advertiser_data["Avg_Budget"][m] / 12 * expected_uplift)
        for m in months
    ])
    prob += total_cost

    # Solve the model
    prob.solve()

    # Store results
    results = []
    for m in months:
        results.append({
            "Month": advertiser_data["Month"][m],
            "Available Agents": A[m].varValue,
            "New Hires": N[m].varValue,
            "Fired": F[m].varValue,
            "Assigned Advertisers": S[m].varValue,
            "Waiting Advertisers": W[m].varValue,
            "Dropped Advertisers": D[m].varValue,
            "Graduated Advertisers": G[m].varValue,
            "Total Cost ($)": round(A[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2),
            "Revenue Uplift ($)": round(S[m].varValue * advertiser_data["Avg_Budget"][m] * expected_uplift, 2),
            "Waiting Cost Penalty ($)": round(W[m].varValue * (advertiser_data["Avg_Budget"][m] / 12 * expected_uplift), 2)
        })

    optimization_results[country] = pd.DataFrame(results)

# Concatenate results into one dataframe
final_optimization_df = pd.concat(optimization_results, names=["Country"]).reset_index(level=1, drop=True).reset_index()




/usr/local/lib/python3.11/dist-packages/pulp/pulp.py:1423: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/usr/local/lib/python3.11/dist-packages/pulp/pulp.py:1423: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
final_optimization_df

,Country,Month,Available Agents,New Hires,Fired,Assigned Advertisers,Waiting Advertisers,Dropped Advertisers,Graduated Advertisers,Total Cost ($),Revenue Uplift ($),Waiting Cost Penalty ($)
0,Canada,2025-01,228.0,0.0,113.0,0.0,371.0,0.0,0.0,3920052.00,0.00,188682.72
1,Canada,2025-02,115.0,0.0,56.0,13.0,703.0,0.0,0.0,1952902.33,80436.92,362481.74
2,Canada,2025-03,59.0,0.0,29.0,8.0,678.0,371.0,0.0,1008507.67,52294.68,369331.18
3,Canada,2025-04,30.0,0.0,2.0,0.0,332.0,703.0,13.0,201498.00,0.00,170442.99
4,Canada,2025-05,41.0,0.0,0.0,0.0,24.0,678.0,8.0,208621.67,0.00,12271.77
...,...,...,...,...,...,...,...,...,...,...,...,...
259,Thailand,2025-08,8.0,0.0,0.0,0.0,155.0,4.0,0.0,16192.67,0.00,14966.61
260,Thailand,2025-09,8.0,3.0,0.0,35.0,137.0,88.0,0.0,16192.67,44599.28,14547.86
261,Thailand,2025-10,11.0,0.0,0.0,105.0,2.0,155.0,0.0,22264.92,159950.70,253.89
262,Thailand,2025-11,11.0,0.0,0.0,0.0,0.0,137.0,35.0,22264.92,0.00,0.00
